In [5]:
!pip install hyperas

In [9]:

import time
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import multi_gpu_model
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice

def data():
    # ----------- import data and perform scaling ----------- #
    df_train = pd.read_csv('data/train.csv')
    df_test = pd.read_csv('data/test.csv')

    X_train = df_train[['xmin', 'ymin', 'xmax', 'ymax']].values
    y_train = df_train[['xloc', 'yloc','zloc']].values

    X_test = df_test[['xmin', 'ymin', 'xmax', 'ymax']].values
    y_test = df_test[['xloc', 'yloc','zloc']].values

    scalar = StandardScaler()
    x_train = scalar.fit_transform(X_train)
    y_train = scalar.fit_transform(y_train)
    x_test = scalar.fit_transform(X_test)
    y_test = scalar.fit_transform(y_test)

    return x_train, y_train, x_test, y_test

def create_model(x_train, y_train, x_test, y_test):
    # ----------- define model ----------- #
    model = Sequential()

    model.add(Dense(6, input_shape=(4,)))

    model.add(Dense({{choice([30,100, 150])}}, activation='relu'))
    model.add(Dense({{choice([30,100, 150])}}, activation='relu'))
    model.add(Dense({{choice([30,100, 150])}}, activation='relu'))
   

    model.add(Dense(3))

#     model = multi_gpu_model(model, gpus=2)
    model.compile(loss='mean_squared_error', metrics=['mae'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})

    # ----------- define callbacks ----------- #
    earlyStopping = EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=7,
								       verbose=1, epsilon=1e-5, mode='min')
    tensorboard = TensorBoard(log_dir="logs/model@{}".format(int(time.time())))

    # ----------- start training ----------- #
    model.fit(x_train, y_train,
              batch_size={{choice([1024, 2048])}},
              epochs=5000,
              callbacks=[tensorboard],
              verbose=1,
              validation_split=0.1)

    # ----------- evaluate model ----------- #
    score, acc = model.evaluate(x_test, y_test, verbose=1)
    print('Test accuracy:', score)

    # ----------- save model and weights ----------- #
    '''
    # NOTE: WAITING ON HYPERAS ISSUE
    model_json = model.to_json()
    modelname = "{}".format(int(time.time()))
    with open("generated_files/model_{}.json".format(modelname), "w") as json_file:
        json_file.write(model_json)
    model.save_weights("generated_files/model_{}.h5".format(modelname))
    print("Saved model to disk")
    '''

    return {'loss': score, 'status': STATUS_OK, 'model': model}


trials = Trials()
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=3,
                                      trials=trials,
                                      eval_space=True,
                                     notebook_name='hyperas')

X_train, Y_train, X_test, Y_test = data()
print("Evaluation of best performing model:", best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)


>>> Imports:
#coding=utf-8

try:
    import time
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense
except:
    pass

try:
    from keras.utils import multi_gpu_model
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [30,100, 150]),
        'Dense_1': hp.choice('Dense_1', [30,100, 150]),
        'Dense_2': hp.choice('Dense_2', [30,100, 150]),
        'optimizer': hp.choice('optimizer', [

job exception: module 'tensorflow.python.framework.ops' has no attribute '_TensorLike'



  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]


AttributeError: module 'tensorflow.python.framework.ops' has no attribute '_TensorLike'

In [10]:
print(keras.__version__)

NameError: name 'keras' is not defined